# 2019 Fantasy Football DSI Contest

### Importing Packages

In [1]:
import pandas as pd
import numpy as np

### Loading Data - Offense

In [14]:
#loading player data
df_train = pd.read_csv("2009stats.csv")
df_train = df_train.append(pd.read_csv("2010stats.csv"))
df_train = df_train.append(pd.read_csv("2011stats.csv"))
df_train = df_train.append(pd.read_csv("2012stats.csv"))
df_train = df_train.append(pd.read_csv("2013stats.csv"))
df_train = df_train.append(pd.read_csv("2014stats.csv"))
df_train = df_train.append(pd.read_csv("2015stats.csv"))
df_train = df_train.append(pd.read_csv("2016stats.csv"))
df_train = df_train.append(pd.read_csv("2017stats.csv"))
df_train = df_train.append(pd.read_csv("2018stats.csv"))
df_train.Team = df_train.Team.replace("STL","LA")
df_train.Team = df_train.Team.replace("SD","LAC")
df_train.Team = df_train.Team.replace("JAC","JAX")

#adding positions
relevant_columns_pos = ["GSIS_ID", "Season", "Pos"]

df_pos = pd.read_csv("2009Pos.csv", usecols = relevant_columns_pos)
df_pos = df_pos.append(pd.read_csv("2010Pos.csv", usecols = relevant_columns_pos))
df_pos = df_pos.append(pd.read_csv("2011Pos.csv", usecols = relevant_columns_pos))
df_pos = df_pos.append(pd.read_csv("2012Pos.csv", usecols = relevant_columns_pos))
df_pos = df_pos.append(pd.read_csv("2013Pos.csv", usecols = relevant_columns_pos))
df_pos = df_pos.append(pd.read_csv("2014Pos.csv", usecols = relevant_columns_pos))
df_pos = df_pos.append(pd.read_csv("2015Pos.csv", usecols = relevant_columns_pos))
df_pos = df_pos.append(pd.read_csv("2016Pos.csv", usecols = relevant_columns_pos))
df_pos = df_pos.append(pd.read_csv("2017Pos.csv", usecols = relevant_columns_pos))
df_pos = df_pos.append(pd.read_csv("2018Pos.csv", usecols = relevant_columns_pos))

#merging positions with offense
df_pos.Pos = df_pos.Pos.replace("FB","RB")

df_offense = pd.merge(df_train, df_pos, left_on = ["playerID", "Season"], 
                      right_on = ["GSIS_ID", "Season"], how = "left").drop("GSIS_ID", axis = 1)

df_offense.Pos = df_offense.groupby("playerID").Pos.fillna(method = 'ffill')
df_offense.Pos = df_offense.groupby("playerID").Pos.fillna(method = 'bfill')

df_offense.dropna(inplace = True)

In [28]:
#fantasy points
df_offense['fantasy_points'] = (df_offense["passyds"] * 0.025 + 4 * df_offense["pass.tds"] - 2 * df_offense["pass.ints"]
                       + 0.1 * df_offense["rushyds"] + 6 * df_offense["rushtds"] + 0.1 * df_offense["recyds"] +
                        6 * df_offense["rec.tds"] + 6 * df_offense["kickret.tds"] + 6 * df_offense["puntret.tds"]
                       + 2 * df_offense["rec.twoptm"] + 2 * df_offense["rush.twoptm"] + 2 * df_offense["pass.twoptm"]
                       + 4 * df_offense["fgm"] - df_offense['fg_pen'] + 
                        df_offense["xpmade"] - 2 * df_offense["fumbslost"])

### Loading Data - Defense

In [3]:
#loading season defense csvs
df_def = pd.read_csv("2009def.csv")
df_def = df_def.append(pd.read_csv("2010def.csv"))
df_def = df_def.append(pd.read_csv("2011def.csv"))
df_def = df_def.append(pd.read_csv("2012def.csv"))
df_def = df_def.append(pd.read_csv("2013def.csv"))
df_def = df_def.append(pd.read_csv("2014def.csv"))
df_def = df_def.append(pd.read_csv("2015def.csv"))
df_def = df_def.append(pd.read_csv("2016def.csv"))
df_def = df_def.append(pd.read_csv("2017def.csv"))
df_def = df_def.append(pd.read_csv("2018def.csv"))

df_def["sum_vals"] = 1

df_abbreviations = pd.read_csv("abbreviations.csv")
df_def = df_def.merge(df_abbreviations, on = ["Tm"])

df_def["Season"] = df_def.groupby("Team")["sum_vals"].transform(lambda x: x.cumsum() + 2008)


df_def.fillna(0, inplace = True)dd

df_def.drop(["Tm", "sum_vals"], inplace = True, axis = 1)

#summing game by game defense stats for each game
df_train['opp'] = df_train.groupby("game.id").Team.transform(lambda x: x.unique()[0])
df_train['opp2'] = df_train.groupby("game.id").Team.transform(lambda x: x.unique()[1])

df_train.loc[df_train['opp'] == df_train['Team'], 'opp'] = df_train.loc[df_train['opp'] == df_train['Team'], 'opp2']
df_train.drop(['opp2'], axis = 1, inplace = True)

df_defense = df_train

df_defense["points_allowed"] = ((df_defense["pass.tds"] + df_defense["rushtds"] + df_defense["kickret.tds"]
                                 + df_defense["puntret.tds"]) * 6 + df_defense["fgm"] * 3 +
                                df_defense["rec.twoptm"] * 2 + df_defense["rush.twoptm"] * 2
                                + df_defense["xpmade"])

df_defense["yards"] = df_defense["passyds"] + df_defense["rushyds"]

df_defense = df_defense.groupby(["Season", "game.id","opp"])["yards", "points_allowed", "pass.ints", "fumbslost"].sum().reset_index()

df_train_sacks = df_train.groupby(["Season", "game.id","Team"])['sacks'].sum().reset_index()

df_defense = pd.merge(df_defense, df_train_sacks, left_on = ["Season", 'game.id', 'opp'],
                     right_on = ['Season', 'game.id', 'Team']).drop('Team', axis = 1)

C:\Users\Tom Bliss\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
#points allowed indicators
df_defense["0pts"] = df_defense.points_allowed == 0
df_defense["1_6pts"] = df_defense.points_allowed.between(1, 6)
df_defense["7_13pts"] = df_defense.points_allowed.between(7, 13)
df_defense["14_17pts"] = df_defense.points_allowed.between(14, 17)
df_defense["18_27pts"] = df_defense.points_allowed.between(18, 27)
df_defense["28_34pts"] = df_defense.points_allowed.between(28, 34)
df_defense["35_45pts"] = df_defense.points_allowed.between(35, 45)
df_defense["45pts"] = df_defense.points_allowed > 45

#yards allowed indicators
df_defense["YA100"] = df_defense.yards < 100
df_defense["YA199"] = df_defense.yards.between(100, 199)
df_defense["YA299"] = df_defense.yards.between(200, 299)
df_defense["YA349"] = df_defense.yards.between(300, 349)
df_defense["YA399"] = df_defense.yards.between(350, 399)
df_defense["YA449"] = df_defense.yards.between(400, 449)
df_defense["YA499"] = df_defense.yards.between(450, 499)
df_defense["YA549"] = df_defense.yards.between(500, 549)
df_defense["YA550"] = df_defense.yards > 549

In [5]:
#doing average per game since no data on game by game TDs / safety
df_def = (df_def.groupby(['Team', 'Season'])[['IntTD', 'KR TD', 'PR TD', 'Sfty', 'OthTD', 'FblTD']].sum()/16).reset_index()

#merging TD and safety data
df_defense = pd.merge(df_defense, df_def, right_on = ["Team", "Season"], left_on = ["opp", "Season"]).drop('opp', axis = 1)



C:\Users\Tom Bliss\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:180: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [6]:
#calculating points per game
df_defense['fantasy_points'] = ((df_defense["IntTD"] + df_defense["KR TD"] + df_defense["PR TD"] + df_defense["OthTD"] +
                       df_defense["FblTD"])*6 + 
                    (df_defense["pass.ints"] + df_defense['fumbslost']) * 2 + 
                    df_defense["sacks"] + df_defense["Sfty"] * 2 +
                    (df_defense["0pts"] + df_defense["YA100"]) * 5 + 
                    df_defense['1_6pts'] * 4 +
                    (df_defense["7_13pts"] + df_defense["YA199"]) * 3 +
                     df_defense["YA299"] * 2 +
                    df_defense["14_17pts"] +
                    (df_defense["YA399"] + df_defense["28_34pts"]) * (-1) +
                    (df_defense["YA449"] + df_defense["35_45pts"]) * (-3) +
                    (df_defense["YA499"] + df_defense["45pts"]) * (-5) +
                    (df_defense["YA549"]) * (-6) +
                    (df_defense["YA550"]) * (-7))

,Season,game.id,yards,points_allowed,pass.ints,fumbslost,sacks,0pts,1_6pts,7_13pts,...,YA549,YA550,Team,IntTD,KR TD,PR TD,Sfty,OthTD,FblTD,points
0,2009,2009091000,330,10,1,1,1.0,False,False,True,...,False,False,PIT,0.1250,0.0,0.0000,0.000,0.0000,0.0625,9.125
1,2009,2009092011,279,17,0,0,1.0,False,False,False,...,False,False,PIT,0.1250,0.0,0.0000,0.000,0.0000,0.0625,5.125
2,2009,2009092701,283,17,0,0,2.0,False,False,False,...,False,False,PIT,0.1250,0.0,0.0000,0.000,0.0000,0.0625,6.125
3,2009,2009100412,270,22,0,1,3.0,False,False,False,...,False,False,PIT,0.1250,0.0,0.0000,0.000,0.0000,0.0625,8.125
4,2009,2009101103,392,14,1,0,7.0,False,False,False,...,False,False,PIT,0.1250,0.0,0.0000,0.000,0.0000,0.0625,10.125
5,2009,2009101805,213,14,2,2,2.0,False,False,False,...,False,False,PIT,0.1250,0.0,0.0000,0.000,0.0000,0.0625,14.125
6,2009,2009102504,423,17,1,1,4.0,False,False,False,...,False,False,PIT,0.1250,0.0,0.0000,0.000,0.0000,0.0625,7.125
7,2009,2009110900,248,4,3,0,2.0,False,True,False,...,False,False,PIT,0.1250,0.0,0.0000,0.000,0.0000,0.0625,15.125
8,2009,2009111504,239,18,0,0,2.0,False,False,False,...,False,False,PIT,0.1250,0.0,0.0000,0.000,0.0000,0.0625,5.125
9,2009,2009112205,316,27,0,1,5.0,False,False,False,...,False,False,PIT,0.1250,0.0,0.0000,0.000,0.0000,0.0625,8.125
